In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from math import sqrt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pickle

In [2]:
fifa21 = pd.read_csv("fifa_new_data.csv")
fifa21.head()

,Unnamed: 0,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,0,242690,https://sofifa.com/player/242690/simon-graves/...,S. Graves,Simon Graves Jensen,20,1999-05-22,191,78,Denmark,...,46+2,45+2,45+2,45+2,46+2,48+2,51+2,51+2,51+2,48+2
1,1,196907,https://sofifa.com/player/196907/danilo-russo/...,D. Russo,Danilo Russo,31,1987-07-08,186,81,Italy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,251238,https://sofifa.com/player/251238/alessio-carlo...,A. Carlone,Alessio Carlone,23,1996-01-20,169,64,Belgium,...,47+2,45+2,45+2,45+2,47+2,45+2,39+2,39+2,39+2,45+2
3,3,244028,https://sofifa.com/player/244028/semi-belkahia...,S. Belkahia,Semi Belkahia,20,1998-12-22,192,87,Germany,...,50+2,50+2,50+2,50+2,50+2,52+2,58+2,58+2,58+2,52+2
4,4,240172,https://sofifa.com/player/240172/eduardo-corti...,Edu Cortina,Eduardo Cortina García,22,1996-09-25,174,64,Spain,...,61+2,62+2,62+2,62+2,61+2,59+2,54+2,54+2,54+2,59+2


In [3]:
fifa21['attacking']= fifa21[list(fifa21.filter(regex='attacking_'))].sum(axis=1)
fifa21['skill']= fifa21[list(fifa21.filter(regex='skill_'))].sum(axis=1)
fifa21['movement']= fifa21[list(fifa21.filter(regex='movement_'))].sum(axis=1)
fifa21['power']= fifa21[list(fifa21.filter(regex='power_'))].sum(axis=1)
fifa21['mentality']= fifa21[list(fifa21.filter(regex='mentality_'))].sum(axis=1)
fifa21['defending']= fifa21[list(fifa21.filter(regex='defending_'))].sum(axis=1)
fifa21['goalkeeping']= fifa21[list(fifa21.filter(regex='goalkeeping_'))].sum(axis=1)
print(fifa21['defending'])


0      156
1       43
2      102
3      166
4      160
      ... 
995    211
996    211
997     39
998    178
999     53
Name: defending, Length: 1000, dtype: int64


In [4]:
#changing height and weight so it fits the model

fifa21['height_cm']= fifa21['height_cm']/2.54
fifa21['weight_kg']= fifa21['weight_kg']*2.205


fifa21['weight_kg']

0      171.990
1      178.605
2      141.120
3      191.835
4      141.120
        ...   
995    189.630
996    174.195
997    171.990
998    165.375
999    165.375
Name: weight_kg, Length: 1000, dtype: float64

In [5]:
#renaming columns

fifa21 = fifa21.rename(columns={'height_cm': 'height', 'value_eur': 'value','weak_foot': 'w/f','skill_moves': 'sm','preferred_foot': 'foot','international_reputation': 'ir',
'pace':'pac', 'shooting':'sho', 'passing':'pas', 'dribbling':'dri', 'physic':'phy'
})

In [6]:
numerical = fifa21._get_numeric_data()
numerical

,Unnamed: 0,sofifa_id,age,height,weight_kg,potential,value,wage_eur,ir,w/f,...,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,attacking,skill,movement,power,mentality,goalkeeping
0,0,242690,20,75.196850,171.990,65,90000,1000,1,3,...,7,6,13,10,172,159,233,244,226,45
1,1,196907,31,73.228346,178.605,62,190000,1000,1,2,...,61,62,63,60,78,80,222,222,140,306
2,2,251238,23,66.535433,141.120,63,140000,1000,1,2,...,10,8,6,9,244,278,335,271,256,43
3,3,244028,20,75.590551,191.835,69,230000,1000,1,4,...,10,8,7,15,208,146,264,286,257,52
4,4,240172,22,68.503937,141.120,75,975000,3000,1,3,...,10,12,11,13,262,320,322,260,361,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,208299,27,74.015748,189.630,70,0,0,1,3,...,10,13,7,15,202,176,291,264,284,60
996,996,179696,30,70.866142,174.195,73,3200000,20000,2,2,...,10,9,8,16,332,350,341,385,426,54
997,997,209830,25,72.440945,171.990,76,3000000,7000,1,2,...,68,62,68,75,99,115,287,217,139,346
998,998,193651,27,69.291339,165.375,59,160000,1000,1,4,...,14,6,9,7,220,278,312,283,328,45


In [7]:
numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 67 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  1000 non-null   int64  
 1   sofifa_id                   1000 non-null   int64  
 2   age                         1000 non-null   int64  
 3   height                      1000 non-null   float64
 4   weight_kg                   1000 non-null   float64
 5   potential                   1000 non-null   int64  
 6   value                       1000 non-null   int64  
 7   wage_eur                    1000 non-null   int64  
 8   ir                          1000 non-null   int64  
 9   w/f                         1000 non-null   int64  
 10  sm                          1000 non-null   int64  
 11  release_clause_eur          935 non-null    float64
 12  team_jersey_number          989 non-null    float64
 13  contract_valid_until        989 no

In [8]:
fifa21['ls']=fifa21['ls'].replace(r'\s+', np.nan, regex=True)
fifa21['lam']=fifa21['lam'].replace(r'\s+', np.nan, regex=True)
fifa21['lcm']=fifa21['lcm'].replace(r'\s+', np.nan, regex=True)
fifa21['ldm']=fifa21['ldm'].replace(r'\s+', np.nan, regex=True)
fifa21['lwb']=fifa21['lwb'].replace(r'\s+', np.nan, regex=True)


In [9]:
fifa21['ls'] = fifa21['ls'].replace(np.nan, '0+0')
fifa21['lam'] = fifa21['lam'].replace(np.nan, '0+0')
fifa21['lcm'] = fifa21['lcm'].replace(np.nan, '0+0')
fifa21['ldm'] = fifa21['ldm'].replace(np.nan, '0+0')
fifa21['lwb'] = fifa21['lwb'].replace(np.nan, '0+0')

In [10]:
fifa21['lwb']

0      46+2
1       0+0
2      47+2
3      50+2
4      61+2
       ... 
995    56+2
996    73+2
997     0+0
998    62+2
999    43+2
Name: lwb, Length: 1000, dtype: object

In [11]:
fifa21['ls']=fifa21['ls'].apply(lambda x: int(x.split('+')[0]) + int(x.split('+')[1]))
fifa21['lam']=fifa21['lam'].apply(lambda x: int(x.split('+')[0]) + int(x.split('+')[1]))
fifa21['lcm']=fifa21['lcm'].apply(lambda x: int(x.split('+')[0]) + int(x.split('+')[1]))
fifa21['lwb']=fifa21['lwb'].apply(lambda x: int(x.split('+')[0]) + int(x.split('+')[1]))
fifa21['ldm']=fifa21['ldm'].apply(lambda x: int(x.split('+')[0]) + int(x.split('+')[1]))
fifa21['ls']

0      39
1       0
2      55
3      47
4      58
       ..
995    43
996    70
997     0
998    49
999    69
Name: ls, Length: 1000, dtype: int64

In [12]:
Cat_OneHotEncoder = fifa21[['foot']]
Cat_OneHotEncoder

,foot
0,Right
1,Right
2,Right
3,Right
4,Right
...,...
995,Right
996,Left
997,Right
998,Left


In [15]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder().fit(Cat_OneHotEncoder)
categoricals_array = encoder.transform(Cat_OneHotEncoder).toarray()

In [17]:
final_columns = list()
to_drop_columns = list()      
         
for columns in encoder.categories_:
    condition = True
    for column in columns.tolist():
        if condition:
            to_drop_columns.append(column)
            final_columns.append(column)
            condition = False
        else:
            final_columns.append(column)

In [18]:
categoricals_df = pd.DataFrame(data = categoricals_array, columns = final_columns)
categoricals_df.columns = map(str.lower, categoricals_df.columns)
categoricals_df

,left,right
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
995,0.0,1.0
996,1.0,0.0
997,0.0,1.0
998,1.0,0.0


In [19]:
fifa21_final = pd.concat([fifa21,categoricals_df], axis = 1)
fifa21_final.head()

,Unnamed: 0,sofifa_id,player_url,short_name,long_name,age,dob,height,weight_kg,nationality,...,rcb,rb,attacking,skill,movement,power,mentality,goalkeeping,left,right
0,0,242690,https://sofifa.com/player/242690/simon-graves/...,S. Graves,Simon Graves Jensen,20,1999-05-22,75.196850,171.990,Denmark,...,51+2,48+2,172,159,233,244,226,45,0.0,1.0
1,1,196907,https://sofifa.com/player/196907/danilo-russo/...,D. Russo,Danilo Russo,31,1987-07-08,73.228346,178.605,Italy,...,NaN,NaN,78,80,222,222,140,306,0.0,1.0
2,2,251238,https://sofifa.com/player/251238/alessio-carlo...,A. Carlone,Alessio Carlone,23,1996-01-20,66.535433,141.120,Belgium,...,39+2,45+2,244,278,335,271,256,43,0.0,1.0
3,3,244028,https://sofifa.com/player/244028/semi-belkahia...,S. Belkahia,Semi Belkahia,20,1998-12-22,75.590551,191.835,Germany,...,58+2,52+2,208,146,264,286,257,52,0.0,1.0
4,4,240172,https://sofifa.com/player/240172/eduardo-corti...,Edu Cortina,Eduardo Cortina García,22,1996-09-25,68.503937,141.120,Spain,...,54+2,59+2,262,320,322,260,361,53,0.0,1.0


In [23]:
#compiling for predicitions
fifa21_TBC = fifa21_final[['age','height', 'attacking','skill','movement','power','mentality','defending','goalkeeping','w/f','sm','ir','pac','sho','pas','dri','phy','ls','lam','lcm','lwb','ldm','left','right'
]]
fifa21_TBC.head().T

,0,1,2,3,4
age,20.00000,31.000000,23.000000,20.000000,22.000000
height,75.19685,73.228346,66.535433,75.590551,68.503937
attacking,172.00000,78.000000,244.000000,208.000000,262.000000
skill,159.00000,80.000000,278.000000,146.000000,320.000000
movement,233.00000,222.000000,335.000000,264.000000,322.000000
power,244.00000,222.000000,271.000000,286.000000,260.000000
mentality,226.00000,140.000000,256.000000,257.000000,361.000000
defending,156.00000,43.000000,102.000000,166.000000,160.000000
goalkeeping,45.00000,306.000000,43.000000,52.000000,53.000000
w/f,3.00000,2.000000,2.000000,4.000000,3.000000


In [24]:
#null values check

pd.set_option('display.max_rows',69)
fifa21_TBC.isna().sum()

age              0
height           0
attacking        0
skill            0
movement         0
power            0
mentality        0
defending        0
goalkeeping      0
w/f              0
sm               0
ir               0
pac            119
sho            119
pas            119
dri            119
phy            119
ls               0
lam              0
lcm              0
lwb              0
ldm              0
left             0
right            0
dtype: int64

In [25]:
#filling null values

mean_pac = fifa21_TBC['pac'].mean()
mean_sho = fifa21_TBC['sho'].mean()
mean_pas = fifa21_TBC['pas'].mean()
mean_dri = fifa21_TBC['dri'].mean()
mean_phy = fifa21_TBC['phy'].mean()

fifa21_TBC['ls']=fifa21_TBC['ls'].replace(0,fifa21_TBC['ls'].mean())
fifa21_TBC['lam']=fifa21_TBC['lam'].replace(0,fifa21_TBC['lam'].mean())
fifa21_TBC['lcm']=fifa21_TBC['lcm'].replace(0,fifa21_TBC['lcm'].mean())
fifa21_TBC['lwb']=fifa21_TBC['lwb'].replace(0,fifa21_TBC['lwb'].mean())
fifa21_TBC['ldm']=fifa21_TBC['ldm'].replace(0,fifa21_TBC['ldm'].mean())

fifa21_TBC['pac'] = fifa21_TBC['pac'].fillna(mean_pac)
fifa21_TBC['sho'] = fifa21_TBC['sho'].fillna(mean_sho)
fifa21_TBC['pas'] = fifa21_TBC['pas'].fillna(mean_pas)
fifa21_TBC['dri'] = fifa21_TBC['dri'].fillna(mean_dri)
fifa21_TBC['phy'] = fifa21_TBC['phy'].fillna(mean_phy)

/var/folders/x0/lydc794d7tjbpf7r4sygzwrw0000gn/T/ipykernel_27746/2358641702.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifa21_TBC['ls']=fifa21_TBC['ls'].replace(0,fifa21_TBC['ls'].mean())
/var/folders/x0/lydc794d7tjbpf7r4sygzwrw0000gn/T/ipykernel_27746/2358641702.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifa21_TBC['lam']=fifa21_TBC['lam'].replace(0,fifa21_TBC['lam'].mean())
/var/folders/x0/lydc794d7tjbpf7r4sygzwrw0000gn/T/ipykernel_27746/2358641702.py:11: SettingWithCopyWarning: 
A valu

In [ ]:
#train test split

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model


In [29]:
# Uploading the previous model
filename = 'fifa_model.sav'
lm = pickle.load(open(filename, 'rb'))

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

predictions = lm.predict(fifa21_TBC)

#R2 = r2_score(y_test, predictions)
#MSE = mean_squared_error(y_test, predictions)
#RMSE = np.sqrt(mean_squared_error(y_test, predictions)) #root_mean_square_error
#MAE = mean_absolute_error(y_test, predictions)

#print(R2)
#print(MSE)
#print(RMSE)
#print(MAE)

In [ ]:
predictions